In [ ]:
import sys
sys.path.append("../")  

In [52]:
import torch
import torch.optim as optim
from torch import nn

import hgp
from hgp.models import HGNNP,CHGNN
from hgp.function import StraightThroughEstimator

import numpy as np
import random
DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
DEVICE


device(type='cuda', index=1)

In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  
np.random.seed(seed) 
random.seed(seed)  
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [54]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 3

weight = 0.93
lr = 4e-3
sub = 0.009
limit = 0.01
h_hyper_prmts["convlayers11"] = {"in_channels": 242, "out_channels": 256, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers14"] = {"in_channels":256, "out_channels": 256, "use_bn": False, "drop_rate": 0.05}
h_hyper_prmts["convlayers141"] = {"in_channels": 256, "out_channels": 256, "use_bn": False, "drop_rate": 0.05}
# h_hyper_prmts["convlayers142"] = {"in_channels": 256, "out_channels": 256, "use_bn": False, "drop_rate": 0.05}
h_hyper_prmts["convlayers143"] = {"in_channels": 256, "out_channels": 256, "use_bn": False, "drop_rate": 0.05}


# l_hyper_prmts["linerlayer113"] = {"in_channels":2048, "out_channels":2048, "use_bn":True, "drop_rate":0.05}
# l_hyper_prmts["linerlayer13"] = {"in_channels":2048, "out_channels":1024, "use_bn":True, "drop_rate":0.05}
# l_hyper_prmts["linerlayer1"] = {"in_channels":1024, "out_channels":512, "use_bn":True, "drop_rate":0.05}
# l_hyper_prmts["linerlayer12334"] = {"in_channels":512, "out_channels":512, "use_bn":True, "drop_rate":0.05}
# l_hyper_prmts["linerlayer12"] = {"in_channels":512, "out_channels":256, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer123"] = {"in_channels":256, "out_channels":128, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer121"] = {"in_channels":128, "out_channels":64, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer31"] = {"in_channels":64, "out_channels":3, "use_bn":True, "drop_rate":0.05}


hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [ ]:
def loss_bs_matrix(outs, hg, device,weight):
   
    H = hg.H.to_dense().to(device)
    outs = outs.to(device)
    nn = torch.matmul(outs, (1 - torch.transpose(outs, 0, 1)))
    ne_k = torch.matmul(nn, H)
    ne_k = ne_k.mul(H)

    H_degree = torch.sum(H, dim=0)
    H_degree = H_degree

    H_1 = ne_k / H_degree
    a2 = 1 - H_1
    a3 = torch.prod(a2, dim=0)
    a3 = a3.sum()
    loss_1 = -1 * a3

    # pun = torch.mul(ne_k, H)

    # loss_1 = pun.sum()
    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)
    loss = weight * loss_1 + loss_2
    return loss, loss_1, loss_2

In [ ]:
class Trainer(nn.Module):

    def __init__(self, net, X, hg, optimizer):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        self.weight = 200
        
    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def run(self, epoch):
        self.train()  
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE,weight=self.weight)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

In [57]:
import hgp.utils
G = hgp.utils.from_pickle_to_hypergraph("../data/primary")
edges, _ = G.e
G.num_e,G.num_v

(12704, 242)

In [58]:
# X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
X = torch.eye(hyper["init_features_dim"])
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None).to(DEVICE)
for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(nn.ReLU().to(DEVICE))
    if v["drop_rate"] > 0:
        hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
hgnn_trainer.layers.append(nn.Softmax(dim=1))

ModuleList(
  (0): HGNNP(
    (layers): ModuleList(
      (0): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.2, inplace=False)
        (theta): Linear(in_features=242, out_features=256, bias=True)
      )
      (1): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=256, out_features=256, bias=True)
      )
      (2): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=256, out_features=256, bias=True)
      )
      (3): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=256, out_features=256, bias=True)
      )
    )
  )
  (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.05, inplace=False)
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): BatchNor

In [59]:
# hgnn_trainer.layers
# for n,p in hgnn_trainer.named_parameters():
#     print(n,p)
hgnn_trainer.weight = weight

In [60]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
optim1 = optim.Adam(hgnn_trainer.parameters(), lr=lr, weight_decay=5e-8)
hgnn_trainer.optimizer = optim1
for epoch in range(20000):
    if hgnn_trainer.weight > limit:
        hgnn_trainer.weight = hgnn_trainer.weight - sub
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        print(f"                , loss1: {temp_loss1.item() / 10}")
        print(f"                , loss2: {temp_loss2.item() / 10}")
        print(f"                , weight: {hgnn_trainer.weight}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: -99.88945922851562
                , loss1: -110.6760498046875
                , loss2: 2.0431819915771485
                , weight: 0.921


in 10 epoch, average loss: -3840.975390625
                , loss1: -4511.032421875
                , loss2: 32.1191162109375
                , weight: 0.831
in 20 epoch, average loss: -6996.7140625
                , loss1: -8992.18046875
                , loss2: 21.40860595703125
                , weight: 0.7409999999999999
in 30 epoch, average loss: -6537.169140625
                , loss1: -9477.25390625
                , loss2: 15.357682800292968
                , weight: 0.6509999999999998
in 40 epoch, average loss: -5726.045703125
                , loss1: -9540.52421875
                , loss2: 12.383376312255859
                , weight: 0.5609999999999997
in 50 epoch, average loss: -4874.952734375
                , loss1: -9554.8640625
                , loss2: 12.305961608886719
                , weight: 0.47099999999999964
in 60 epoch, average loss: -4016.80546875
                , loss1: -9559.06875
                , loss2: 12.323222351074218
                , weight: 0.380999

KeyboardInterrupt: 

In [61]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

4070

In [63]:
num_nodes = outs_straight.sum(dim=0)
print(num_nodes)
(torch.max(num_nodes).item() - torch.min(num_nodes).item()) / num_nodes.sum().item()


tensor([81., 80., 81.], device='cuda:1', grad_fn=<SumBackward1>)


0.004132231404958678